# ラベリング

In [ ]:
col_category = []
new_col_category = []
col = X.columns
for column in col:
    
    if X[column].dtypes == object or X[column].dtypes == bool :
        #print(column)
        col_category.append(column)
        #Missing
        X[column] = X[column].fillna('NaN')
        X_test[column] = X_test[column].fillna('NaN')
        
        #label encoding
        ##map
        class_map = {label: idx for idx, label in enumerate(np.unique(X[column]))}
        
        ##mapping
        X[f'{column}_label'] = X[column].map(class_map)
        X_test[f'{column}_label'] = X_test[column].map(class_map)
        
        new_col_category.append(f'{column}_label')    
        
        #frequency encoding
        ##map
        freq_map = X[column].value_counts()
        ##mapping
        X[f'{column}_freq'] = X[column].map(freq_map)
        X_test[f'{column}_freq'] = X_test[column].map(freq_map)
        
        new_col_category.append(f'{column}_freq')
        
        ##Standard
        ss = StandardScaler()
        X[f'{column}_freq'] = ss.fit_transform(X[f'{column}_freq'].values.reshape(-1, 1))
        X_test[f'{column}_freq'] = ss.transform(X_test[f'{column}_freq'].values.reshape(-1, 1))

# targeting

In [ ]:
for c in col_category:
    
    data_tmp = pd.DataFrame({c:X[c], 'target':target})
    target_mean = data_tmp.groupby(c)['target'].mean()
    
    #X_test[f'{c}_target'] = X_test[c].map(target_mean)
    X_test[f'{c}_target_log'] = np.log(X_test[c].map(target_mean))
    tmp = np.repeat(np.nan, X.shape[0])
    
    kf = KFold(n_splits=4, shuffle=True, random_state=72)
    
    for idx_1, idx_2 in kf.split(X):
        
        target_mean = data_tmp.iloc[idx_1].groupby(c)['target'].mean()
        
        tmp[idx_2] = X[c].iloc[idx_2].map(target_mean)
        
    #X[f'{c}_target'] = tmp
    X[f'{c}_target_log'] = np.log(tmp)
    
    #new_col_category.append(f'{c}_target')
    new_col_category.append(f'{c}_target_log')